In [1]:
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import os

In [2]:
load_dotenv()

True

In [3]:
clients = pd.read_csv(os.getenv('DATA_PATH') + '/' + 'tj38.csv', encoding='ISO-8859-1', sep=';')
poste =  pd.read_csv(os.getenv('DATA_PATH') + '/' + 'tj24.csv', encoding='ISO-8859-1', sep=';')

In [4]:
clients.shape

(2159265, 7)

In [5]:
poste.shape

(859128, 2)

In [6]:
poste.head(1)

,COMAX,COPOST
0,001932f4aa,66420


In [7]:
clients.head(1)

,COMAX,AGE,COSEXE,CTSCPI,CESITC,DDVALE,DFVALE
0,6e3a2b9fa1,55,F,4600,1,2013-02-12,9999-01-01


In [8]:
data = pd.merge(clients,poste,on='COMAX',how='left')

In [9]:
data.head()

,COMAX,AGE,COSEXE,CTSCPI,CESITC,DDVALE,DFVALE,COPOST
0,6e3a2b9fa1,55,F,4600,1,2013-02-12,9999-01-01,34170
1,1b44a67f61,41,M,4700,1,2019-05-29,9999-01-01,34490
2,1b44a67f61,41,M,4700,0,2019-02-07,2019-03-21,34490
3,1b44a67f61,41,M,4700,0,2019-02-04,2019-02-05,34490
4,1b44a67f61,41,M,4700,0,2019-03-21,2019-05-29,34490


## Corp The Data

In [10]:
# delete duplicated matricules
data = data.drop_duplicates(subset="COMAX")

In [11]:
# in some cases there was an input mistake
# so instead of typing the age they type the
# birth year, so we correct that
data = data.replace(2019, 2021-2019)

In [12]:
# take only the age less or equal to 110 for demographoc statistc reasons
data = data[data['AGE'] <= 110]

In [13]:
# get the list of miss typed data
lol = list()
for i in data['CTSCPI'].unique():
    if not i.isdigit():
        lol.append(i)

# replce the missed data, and miss created one with nan values
data['CTSCPI'].replace(lol, np.nan, inplace=True)

In [14]:
# add the zero, since in the corresponding table of topologies
# it is an unkown category
data['CTSCPI'].replace(0., np.nan, inplace=True)

# delete the rows with nan values
data.dropna(subset=['CTSCPI'], inplace=True)

In [15]:
data["CTSCPI"] = data["CTSCPI"].astype(int)

In [16]:
# we replace the sex with numbers, 0 for M and 1 for F
data['COSEXE'].replace('M', 0, inplace=True)
data['COSEXE'].replace('F', 1, inplace=True)

In [17]:
data.shape

(831367, 8)

In [18]:
data.head()

,COMAX,AGE,COSEXE,CTSCPI,CESITC,DDVALE,DFVALE,COPOST
0,6e3a2b9fa1,55,1,4600,1,2013-02-12,9999-01-01,34170
1,1b44a67f61,41,0,4700,1,2019-05-29,9999-01-01,34490
6,7e33583438,74,1,8600,0,2011-12-22,2017-10-16,73100
9,b65dd1ba1d,41,0,8400,0,2011-12-22,2017-09-13,
10,bdf1f63a98,32,1,3500,1,2017-12-14,9999-01-01,79800


In [19]:
lol = list()

for i in data['COPOST'].unique():
    if not str(i).isdigit():
        lol.append(i)

In [20]:
# replce the missed data, and miss created one with nan values
data['COPOST'].replace(lol, np.nan, inplace=True)

# delete the rows with nan values
data.dropna(subset=['COPOST'], inplace=True)

In [21]:
data.shape

(656954, 8)

In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 656954 entries, 0 to 2159265
Data columns (total 8 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   COMAX   656954 non-null  object
 1   AGE     656954 non-null  int64 
 2   COSEXE  656954 non-null  int64 
 3   CTSCPI  656954 non-null  int64 
 4   CESITC  656954 non-null  int64 
 5   DDVALE  656954 non-null  object
 6   DFVALE  656954 non-null  object
 7   COPOST  656954 non-null  object
dtypes: int64(4), object(4)
memory usage: 45.1+ MB


In [23]:
data["COPOST"] = data["COPOST"].astype(int)

In [24]:
data.shape

(656954, 8)

## Save The Data

In [25]:
data = data[['COMAX', 'AGE', 'COSEXE', 'CTSCPI']]

In [26]:
data.to_csv(os.getenv('CLEANED_DATA_PATH') + '/' + 'TJ38.csv', index=False, sep="\t")